In [ ]:
# | default_exp application

In [ ]:
# | export

from typing import *
from typing import get_type_hints

from enum import Enum
from pathlib import Path
import json
import yaml
from copy import deepcopy
from os import environ
from datetime import datetime, timedelta
import tempfile
from contextlib import contextmanager, asynccontextmanager
import time
from inspect import signature, iscoroutinefunction
import functools

from fastcore.foundation import patch

import anyio
import asyncio
from asyncio import iscoroutinefunction  # do not use the version from inspect
import httpx
from fastapi import FastAPI
from fastapi import status, Depends, HTTPException, Request, Response
from fastapi.openapi.docs import get_swagger_ui_html, get_redoc_html
from fastapi.openapi.utils import get_openapi
from fastapi.responses import FileResponse, RedirectResponse
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
from pydantic import Field, HttpUrl, EmailStr, PositiveInt
from pydantic.schema import schema
from pydantic.json import timedelta_isoformat
from aiokafka import AIOKafkaProducer

import confluent_kafka
from confluent_kafka import Producer
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka import Message, KafkaError
import asyncer

import fast_kafka_api._components.logger

fast_kafka_api._components.logger.should_supress_timestamps = True

import fast_kafka_api
from fast_kafka_api._components.aiokafka_loop import aiokafka_consumer_loop
from fast_kafka_api._components.asyncapi import (
    KafkaMessage,
    export_async_spec,
    ConsumeCallable,
    ProduceCallable,
    KafkaBroker,
    ContactInfo,
    KafkaServiceInfo,
    KafkaBrokers,
)
from fast_kafka_api._components.logger import get_logger, supress_timestamps

[INFO] fast_kafka_api._components.asyncapi: ok


In [ ]:
# | export
logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
import pytest
import yaml
import unittest.mock
from dataclasses import dataclass
import shutil

import nest_asyncio

import uvicorn
from fastapi.testclient import TestClient
from starlette.datastructures import Headers

from rich.pretty import pprint

from fast_kafka_api.testing import true_after, mock_AIOKafkaProducer_send

In [ ]:
# | notest
# allows async calls in notebooks
import nest_asyncio

nest_asyncio.apply()

In [ ]:
kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
    "bootstrap_servers": f"{kafka_server_url}:{kafka_server_port}",
    "group_id": f"{kafka_server_url}:{kafka_server_port}_group",
    "auto_offset_reset": "earliest",
}

In [ ]:
# | export


class FastKafkaAPI(FastAPI):
    def __init__(
        self,
        *,
        title: str = "FastKafkaAPI",
        kafka_config: Dict[str, Any],
        contact: Optional[Dict[str, Union[str, Any]]] = None,
        kafka_brokers: Optional[Dict[str, Any]] = None,
        root_path: Optional[Union[Path, str]] = None,
        **kwargs,
    ):
        """  Combined REST and Kafka service
        
        Params:
            title: name of the service, used for generating documentation
            kafka_config: 
            contact:
            kafka_brokers:
            root_path:
            kwargs: parameters passed to FastAPI constructor
        """
        self._kafka_config = kafka_config

        config_defaults = {
            "bootstrap_servers": "localhost:9092",
            "auto_offset_reset": "earliest",
            "max_poll_records": 100,
            "max_buffer_size": 100,
        }

        for key, value in config_defaults.items():
            if key not in kafka_config:
                kafka_config[key] = value

        if root_path is None:
            root_path = Path(".")
        self._root_path = Path(root_path)

        if kafka_brokers is None:
            kafka_brokers = {
                "localhost": KafkaBroker(
                    url="https://localhost",
                    description="Local (dev) Kafka broker",
                    port="9092",
                )
            }
        if contact is None:
            contact = dict(
                name="author", url="https://www.google.com", email="noreply@gmail.com"
            )

        super().__init__(title=title, contact=contact, **kwargs)

        self._consumers_store: Dict[str, Tuple[ConsumeCallable, Dict[str, Any]]] = {}

        self._producers_store: Dict[
            str, Tuple[ProduceCallable, AIOKafkaProducer, Dict[str, Any]]
        ] = {}
        self._on_error_topic: Optional[str] = None

        contact_info = ContactInfo(**contact)  # type: ignore
        self._kafka_service_info = KafkaServiceInfo(
            title=self.title,
            version=self.version,
            description=self.description,
            contact=contact_info,
        )

        self._kafka_brokers = KafkaBrokers(brokers=kafka_brokers)

        self._asyncapi_path = self._root_path / "asyncapi"
        (self._asyncapi_path / "docs").mkdir(exist_ok=True, parents=True)
        (self._asyncapi_path / "spec").mkdir(exist_ok=True, parents=True)
        self.mount(
            "/asyncapi",
            StaticFiles(directory=self._asyncapi_path / "docs"),
            name="asyncapi",
        )

        self._is_shutting_down: bool = False
        self._kafka_consumer_tasks: List[asyncio.Task[Any]] = []
        self._kafka_producer_tasks: List[asyncio.Task[Any]] = []

        @self.get("/", include_in_schema=False)
        def redirect_root_to_asyncapi():
            return RedirectResponse("/asyncapi")

        @self.get("/asyncapi", include_in_schema=False)
        async def redirect_asyncapi_docs():
            return RedirectResponse("/asyncapi/index.html")

        @self.get("/asyncapi.yml", include_in_schema=False)
        async def download_asyncapi_yml():
            return FileResponse(self._asyncapi_path / "spec" / "asyncapi.yml")

        @self.on_event("startup")
        async def on_startup(app=self):
            await app._on_startup()

        @self.on_event("shutdown")
        async def on_shutdown(app=self):
            await app._on_shutdown()

    async def _on_startup(self) -> None:
        raise NotImplementedError

    async def _on_shutdown(self) -> None:
        raise NotImplementedError

    def consumes(
        self,
        topic: Optional[str] = None,
        *,
        prefix: str = "on_",
        **kwargs,
    ) -> ConsumeCallable:
        raise NotImplementedError

    def produces(
        self,
        topic: Optional[str] = None,
        *,
        prefix: str = "to_",
        producer: AIOKafkaProducer = None,
        **kwargs,
    ) -> ProduceCallable:
        raise NotImplementedError

In [ ]:
# | export


def _get_topic_name(
    topic_callable: Union[ConsumeCallable, ProduceCallable], prefix: str = "on_"
) -> str:
    topic = topic_callable.__name__
    if not topic.startswith(prefix) or len(topic) <= len(prefix):
        raise ValueError(f"Function name '{topic}' must start with {prefix}")
    topic = topic[len(prefix) :]

    return topic

In [ ]:
def on_topic_name_1():
    pass

assert _get_topic_name(on_topic_name_1) == "topic_name_1"

assert _get_topic_name(on_topic_name_1, prefix="on_topic_") == "name_1"

In [ ]:
def create_testing_app():
    root_path="/tmp/000_FastKafkaAPI"
    if Path(root_path).exists():
        shutil.rmtree(root_path)
        
    app = FastKafkaAPI(
        kafka_brokers={
            "local": {
                "url": "kafka",
                "name": "development",
                "description": "Local (dev) Kafka broker",
                "port": 9092,
            }
        },
        kafka_config=kafka_config,
        root_path=root_path,
    )

    return app

In [ ]:
app = create_testing_app()

In [ ]:
# | export


@patch
def consumes(
    self: FastKafkaAPI,
    topic: Optional[str] = None,
    *,
    prefix: str = "on_",
    **kwargs,
) -> ConsumeCallable:
    """Decorator registering the callback called when a message is received in a topic.

    This function decorator is also responsible for registering topics for AsyncAPI specificiation and documentation.

    Params:
        topic: Kafka topic that the consumer will subscribe to and execute the decorated function when it receives a message from the topic, default: None
            If the topic is not specified, topic name will be inferred from the decorated function name by stripping the defined prefix
        prefix: Prefix stripped from the decorated function to define a topic name if the topic argument is not passed, default: "on_"
            If the decorated function name is not prefixed with the defined prefix and topic argument is not passed, then this method will throw ValueError
        **kwargs: Keyword arguments that will be passed to AIOKafkaConsumer, used to configure the consumer

    Returns:
        A function returning the same function

    Throws:
        ValueError

    """

    def _decorator(
        on_topic: ConsumeCallable,
        topic: str = topic,
        kwargs=kwargs
    ) -> ConsumeCallable:
        if topic is None:
            topic = _get_topic_name(topic_callable=on_topic, prefix=prefix)
            
        self._consumers_store[topic] = (on_topic, kwargs)

        return on_topic

    return _decorator

In [ ]:
app = create_testing_app()

# Basic check
@app.consumes()
def on_my_topic_1(msg: BaseModel) -> None:
    pass

assert app._consumers_store["my_topic_1"] == (on_my_topic_1, {}), app._consumers_store

# Check topic setting
@app.consumes(topic="test_topic_2")
def some_func_name(msg: BaseModel) -> None:
    pass

assert app._consumers_store["test_topic_2"] == (some_func_name, {}), app._consumers_store

# Check prefix change
@app.consumes(prefix="for_")
def for_test_topic_3(msg: BaseModel) -> None:
    pass

assert app._consumers_store["test_topic_3"] == (for_test_topic_3, {}), app._consumers_store

# Check passing of kwargs
kwargs = {"arg1": "val1", "arg2": 2}
@app.consumes(topic="test_topic", **kwargs)
def for_test_kwargs(msg: BaseModel):
    pass

assert app._consumers_store["test_topic"] == (for_test_kwargs, kwargs), app._consumers_store

In [ ]:
# | export


def produce_decorator(self: FastKafkaAPI, func: ProduceCallable, topic: str):
    @functools.wraps(func)
    async def _produce_async(*args, **kwargs):
        return_val = await func(*args, **kwargs)
        _, producer, _ = self._producers_store[topic]
        fut = await producer.send(topic, return_val.json().encode("utf-8"))
        msg = await fut
        return return_val
    
    @functools.wraps(func)
    def _produce_sync(*args, **kwargs):
        return_val = func(*args, **kwargs)
        _, producer, _ = self._producers_store[topic]
        asyncio.create_task(producer.send(topic, return_val.json().encode("utf-8")))
        return return_val

    return _produce_async if iscoroutinefunction(func) else _produce_sync

In [ ]:
@contextmanager
def mock_AIOKafkaProducer_send():
    with unittest.mock.patch("__main__.AIOKafkaProducer.send") as mock:

        async def _f():
            pass

        mock.return_value = asyncio.create_task(_f())

        yield mock

In [ ]:
async def test_me():
    with mock_AIOKafkaProducer_send() as send_mock:
        topic="test_topic"

        class MockMsg(BaseModel):
            id = 123

        def test_func(mock_msg: MockMsg):
            return mock_msg

        producer = AIOKafkaProducer()
        app = unittest.mock.Mock()
        app._producers_store = {topic: (test_func, producer, {})}

        test_func = produce_decorator(app, test_func, topic)

        mock_msg = MockMsg()
        value = test_func(mock_msg)
        await producer.stop()

        send_mock.assert_called_once_with(topic, mock_msg.json().encode("utf-8"))
        assert iscoroutinefunction(test_func) == False
        assert value == mock_msg
        
asyncio.run(test_me())

In [ ]:
async def test_me():
    with mock_AIOKafkaProducer_send() as send_mock:
        topic="test_topic"

        class MockMsg(BaseModel):
            id = 123

        async def test_func(mock_msg: MockMsg):
            return mock_msg

        producer = AIOKafkaProducer()
        app = unittest.mock.Mock()
        app._producers_store = {topic: (test_func, producer, {})}

        test_func = produce_decorator(app, test_func, topic)

        mock_msg = MockMsg()
        value = await test_func(mock_msg)
        await producer.stop()

        send_mock.assert_called_once_with(topic, mock_msg.json().encode("utf-8"))
        assert iscoroutinefunction(test_func) == True
        assert value == mock_msg
        
asyncio.run(test_me())

In [ ]:
# | export


@patch
def produces(
    self: FastKafkaAPI,
    topic: Optional[str] = None,
    *,
    prefix: str = "to_",
    producer: AIOKafkaProducer = None,
    **kwargs,
) -> ProduceCallable:
    """Decorator registering the callback called when delivery report for a produced message is received

    This function decorator is also responsible for registering topics for AsyncAPI specificiation and documentation.

    Params:
        topic: Kafka topic that the producer will send returned values from the decorated function to, default: None
            If the topic is not specified, topic name will be inferred from the decorated function name by stripping the defined prefix
        prefix: Prefix stripped from the decorated function to define a topic name if the topic argument is not passed, default: "to_"
            If the decorated function name is not prefixed with the defined prefix and topic argument is not passed, then this method will throw ValueError
        producer:
        **kwargs: Keyword arguments that will be passed to AIOKafkaProducer, used to configure the producer

    Returns:
        A function returning the same function

    Throws:
        ValueError

    """
    def _decorator(
        on_topic: ProduceCallable,
        topic: str = topic,
        kwargs=kwargs
    ) -> ProduceCallable:
        if topic is None:
            topic = _get_topic_name(topic_callable=on_topic, prefix=prefix)
            
        self._producers_store[topic] = (on_topic, producer, kwargs)
        
        return produce_decorator(self, on_topic, topic)

    return _decorator

In [ ]:
app = create_testing_app()

# Basic check
def to_my_topic_1(msg: BaseModel) -> None:
    pass

# Must be done without sugar to keep the original function reference
check_func = to_my_topic_1
to_my_topic_1 = app.produces()(to_my_topic_1)

assert app._producers_store["my_topic_1"] == (check_func, None, {}), f"{app._producers_store}, {to_my_topic_1}"

# Check topic setting
def some_func_name(msg: BaseModel) -> None:
    pass

check_func = some_func_name
some_func_name = app.produces(topic="test_topic_2")(some_func_name)

assert app._producers_store["test_topic_2"] == (check_func, None, {}), app._producers_store

# Check prefix change
@app.produces(prefix="for_")
def for_test_topic_3(msg: BaseModel) -> None:
    pass

check_func = for_test_topic_3
some_func_name = app.produces(prefix="for_")(for_test_topic_3)

assert app._producers_store["test_topic_3"] == (check_func, None, {}), app._producers_store

# Check passing of kwargs
kwargs = {"arg1": "val1", "arg2": 2}
def for_test_kwargs(msg: BaseModel):
    pass

check_func = for_test_kwargs
for_test_kwargs = app.produces(topic="test_topic", **kwargs)(for_test_kwargs)

assert app._producers_store["test_topic"] == (check_func, None, kwargs), app._producers_store

# Check passing of custom Producer
async def test_me():
    kwargs = {"arg1": "val1", "arg2": 2}
    def for_test_producer(msg: BaseModel):
        pass

    check_func = for_test_producer
    producer = AIOKafkaProducer()
    for_test_producer = app.produces(topic="test_producer", producer=producer, **kwargs)(for_test_producer)

    assert app._producers_store["test_producer"] == (check_func, producer, kwargs), app._producers_store

await test_me()

In [ ]:
class MyInfo(BaseModel):
    mobile: str = Field(..., example="+385987654321")
    name: str = Field(..., example="James Bond")


class MyMsgUrl(BaseModel):
    info: MyInfo = Field(..., example=dict(mobile="+385987654321", name="James Bond"))
    url: HttpUrl = Field(..., example="https://sis.gov.uk/agents/007")


class MyMsgEmail(BaseModel):
    msg_url: MyMsgUrl = Field(
        ...,
        example=dict(
            info=dict(mobile="+385987654321", name="James Bond"),
            url="https://sis.gov.uk/agents/007",
        ),
    )
    email: EmailStr = Field(..., example="agent-007@sis.gov.uk")


def setup_testing_app():
    app = create_testing_app()

    @app.consumes("my_topic_1")
    def on_my_topic_one(msg: MyMsgUrl):
        logger.debug(f"on_my_topic_one(msg={msg},)")

    @app.consumes()
    async def on_my_topic_2(msg: MyMsgEmail):
        logger.debug(f"on_my_topic_2(msg={msg},)")

    with pytest.raises(ValueError) as e:
        @app.consumes()
        def my_topic_3(msg: MyMsgEmail):
            raise NotImplemented

    @app.produces()
    def to_my_topic_3(url: str) -> MyMsgUrl:
        logger.debug(f"on_my_topic_3(msg={url}")
        return MyMsgUrl(info=MyInfo("+38512345678", "Sean Connery"), url=url)

    @app.produces()
    async def to_my_topic_4(msg: MyMsgEmail) -> MyMsgEmail:
        logger.debug(f"on_my_topic_4(msg={msg}")
        return msg

    return app

In [ ]:
app = setup_testing_app()

assert set(app._consumers_store.keys()) == set(["my_topic_1", "my_topic_2"])
assert set(app._producers_store.keys()) == set(["my_topic_3", "my_topic_4"])

print(f"app._kafka_service_info={app._kafka_service_info}")
print(f"app._kafka_brokers={app._kafka_brokers}")

app._kafka_service_info=title='FastKafkaAPI' version='0.1.0' description='' contact=ContactInfo(name='author', url=HttpUrl('https://www.google.com', ), email='noreply@gmail.com')
app._kafka_brokers=brokers={'local': KafkaBroker(url='kafka', description='Local (dev) Kafka broker', port='9092', protocol='kafka', security=None)}


In [ ]:
# | export

def populate_consumers(
    *,
    app: FastKafkaAPI,
    is_shutting_down_f: Callable[[], bool],
) -> List[asyncio.Task]:
    config: Dict[str, Any] = app._kafka_config
    tx = [
        asyncio.create_task(
            aiokafka_consumer_loop(
                topics=[topic],
                callbacks={topic: consumer},
                msg_types={topic: signature(consumer).parameters['msg'].annotation},
                is_shutting_down_f=is_shutting_down_f,
                **{**config, **configuration}
            )
        )
        for topic, (consumer, configuration) in app._consumers_store.items()
    ]

    return tx

In [ ]:
# | export

# TODO: Add passing of vars

async def populate_producers(
    *,
    app: FastKafkaAPI
) -> None:
    config: Dict[str, Any] = app._kafka_config
    for topic, (topic_callable, potential_producer, configuration) in app._producers_store.items():
        if type(potential_producer) != AIOKafkaProducer:
            producer = AIOKafkaProducer(
                **{"bootstrap_servers": config["bootstrap_servers"], **configuration}
            )
            asyncio.create_task(producer.start())
            app._producers_store[topic] = (topic_callable, producer, configuration)
        else:
            asyncio.create_task(potential_producer.start())

In [ ]:
# | export


@patch
async def _on_startup(self: FastKafkaAPI) -> None:
    export_async_spec(
        consumers={topic:topic_callable for topic, (topic_callable, _) in self._consumers_store.items()},  # type: ignore
        producers={topic:topic_callable for topic, (topic_callable, _, _) in self._producers_store.items()},  # type: ignore
        kafka_brokers=self._kafka_brokers,
        kafka_service_info=self._kafka_service_info,
        asyncapi_path=self._asyncapi_path,
    )

    self._is_shutting_down = False

    def is_shutting_down_f(self: FastKafkaAPI = self) -> bool:
        return self._is_shutting_down

    self._kafka_consumer_tasks = populate_consumers(
        app=self,
        is_shutting_down_f=is_shutting_down_f,
    )
    
    await populate_producers(app=self)


@patch
async def _on_shutdown(self: FastKafkaAPI) -> None:
    self._is_shutting_down = True
    
    if self._kafka_consumer_tasks:
        await asyncio.wait(self._kafka_consumer_tasks)
    
    [await producer.stop() for _, producer, _ in self._producers_store.values()]
    
    self._is_shutting_down = False

In [ ]:
@asynccontextmanager
async def start_test_app():
    app = setup_testing_app()
    try:
        await app._on_startup()
        yield app
    finally:
        await app._on_shutdown()

In [ ]:
expected = """asyncapi: 2.5.0
info:
  title: FastKafkaAPI
  version: 0.1.0
  description: ''
  contact:
    name: author
    url: https://www.google.com
    email: noreply@gmail.com
servers:
  local:
    url: kafka
    description: Local (dev) Kafka broker
    protocol: kafka
    variables:
      port:
        default: '9092'
channels:
  my_topic_1:
    subscribe:
      message:
        $ref: '#/components/messages/MyMsgUrl'
  my_topic_2:
    subscribe:
      message:
        $ref: '#/components/messages/MyMsgEmail'
  my_topic_3:
    publish:
      message:
        $ref: '#/components/messages/MyMsgUrl'
  my_topic_4:
    publish:
      message:
        $ref: '#/components/messages/MyMsgEmail'
components:
  messages:
    MyMsgUrl:
      payload:
        title: MyMsgUrl
        type: object
        properties:
          info:
            title: Info
            example:
              mobile: '+385987654321'
              name: James Bond
            allOf:
            - $ref: '#/components/schemas/MyInfo'
          url:
            title: Url
            example: https://sis.gov.uk/agents/007
            minLength: 1
            maxLength: 2083
            format: uri
            type: string
        required:
        - info
        - url
        example:
          info:
            mobile: '+385987654321'
            name: James Bond
          url: https://sis.gov.uk/agents/007
    MyMsgEmail:
      payload:
        title: MyMsgEmail
        type: object
        properties:
          msg_url:
            title: Msg Url
            example:
              info:
                mobile: '+385987654321'
                name: James Bond
              url: https://sis.gov.uk/agents/007
            allOf:
            - $ref: '#/components/messages/MyMsgUrl'
          email:
            title: Email
            example: agent-007@sis.gov.uk
            type: string
            format: email
        required:
        - msg_url
        - email
        example:
          msg_url:
            info:
              mobile: '+385987654321'
              name: James Bond
            url: https://sis.gov.uk/agents/007
          email: agent-007@sis.gov.uk
  schemas:
    MyInfo:
      payload:
        title: MyInfo
        type: object
        properties:
          mobile:
            title: Mobile
            example: '+385987654321'
            type: string
          name:
            title: Name
            example: James Bond
            type: string
        required:
        - mobile
        - name
  securitySchemes: {}
"""

In [ ]:
async def test_me():
    async with start_test_app() as app:
        client = TestClient(app)
        response = client.get("/asyncapi.yml")
        assert response.status_code == 200
        assert yaml.safe_load(response.text) == yaml.safe_load(
            expected
        ), f"{yaml.safe_load(response.text)} != {yaml.safe_load(expected)}"

asyncio.run(test_me())
print("ok")

[INFO] fast_kafka_api._components.asyncapi: Async specifications generated at: '/tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api._components.asyncapi: Async docs generated at '/tmp/000_FastKafkaAPI/asyncapi/docs'
[INFO] fast_kafka_api._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag /tmp/000_FastKafkaAPI/asyncapi/spec/asyncapi.yml @asyncapi/html-template -o /tmp/000_FastKafkaAPI/asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/000_FastKafkaAPI/asyncapi/docs.


[INFO] fast_kafka_api._components.aiokafka_loop: Consumer created.
[INFO] fast_kafka_api._components.aiokafka_loop: Consumer created.
[INFO] fast_kafka_api._components.aiokafka_loop: Consumer started.
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'my_topic_1'})
[INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'my_topic_1'}
[INFO] fast_kafka_api._components.aiokafka_loop: Consumer subscribed.
[INFO] f

In [ ]:
# don't wait for specs to be generated (takes 10 sec or so)
with unittest.mock.patch("__main__.export_async_spec"):
    
    # mock up send method of AIOKafkaProducer
    with mock_AIOKafkaProducer_send() as mock:

        app = create_testing_app()
        
        @app.produces()
        async def to_my_test_topic(mobile: str, url: str) -> MyMsgUrl:
            msg = MyMsgUrl(
                info=dict(mobile=mobile, name="James Bond"), url=url
            )
            return msg

        try:
            await app._on_startup()
            await to_my_test_topic(mobile="+385912345678", url="https://www.vip.hr")
        finally:
            await app._on_shutdown()

        mock.assert_called_with(
            "my_test_topic",
            b'{"info": {"mobile": "+385912345678", "name": "James Bond"}, "url": "https://www.vip.hr"}',
        )        